https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504

In [33]:
# Topic model
from bertopic import BERTopic# Dimension reduction
from umap import UMAP

In [34]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
import pickle

In [36]:
identified_repos = from_disk(grtr)
t_name = 't2.3.1s'

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/identified_repos_v1.pkl


In [37]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.3.1s.pkl


In [38]:
identified_repos_topics = pd.merge(identified_repos, testset[["name", "topics"]], on="name", how="left")

In [39]:
identified_repos_topics

,name,tag,topics
0,0x4D31/awesome-threat-detection,secumon,connect logic spark trace instrument dispatch ...
1,0xN3utr0n/Kanis,secumon,viruses elf sense information anomaly process ...
2,airbnb/binaryalert,secumon,aws threat match read status fire badge bucket...
3,airbnb/streamalert,secumon,research aws automate source logic log support...
4,ait-aecid/logdata-anomaly-miner,secumon,raw information anomaly wiki deployment c ca d...
...,...,...,...
115,arch3rPro/PentestTools,pentest,connect page postgresql servers exe error java...
116,whiterabb17/sifter,pentest,attack penetration code request information ex...
117,hackThacker/advtools,pentest,attack pin code qr information website reverse...
118,rcallaby/Hacking-Study-Guide,pentest,attack systems windows role personal penetrati...


In [40]:
identified_repos_topics[identified_repos_topics['topics'].notna()]

,name,tag,topics
0,0x4D31/awesome-threat-detection,secumon,connect logic spark trace instrument dispatch ...
1,0xN3utr0n/Kanis,secumon,viruses elf sense information anomaly process ...
2,airbnb/binaryalert,secumon,aws threat match read status fire badge bucket...
3,airbnb/streamalert,secumon,research aws automate source logic log support...
4,ait-aecid/logdata-anomaly-miner,secumon,raw information anomaly wiki deployment c ca d...
...,...,...,...
115,arch3rPro/PentestTools,pentest,connect page postgresql servers exe error java...
116,whiterabb17/sifter,pentest,attack penetration code request information ex...
117,hackThacker/advtools,pentest,attack pin code qr information website reverse...
118,rcallaby/Hacking-Study-Guide,pentest,attack systems windows role personal penetrati...


https://umap-learn.readthedocs.io/en/latest/parameters.html

In [41]:
testset['topics']

0         code program process demo colour c debug notic...
1         process box generate efficient partition merge...
2         attack pin code store compatibility authentica...
3         html q youtube space api match access micropho...
4         bank server free admin help list xray head fun...
                                ...                        
221966    software systems hackerrank bank applications ...
221967    index liquibase driver page t5 trace app sprin...
221968    connect code acl request information authentic...
221969    api ml request core machine context arkit av d...
221970    connect elements case servers documentation sc...
Name: topics, Length: 218335, dtype: object

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#cr_start()
#word_doc_matrix = vectorizer.fit_transform(testset['topics'])
#runtime_worddoc = cr_getrt()
# Initiate UMAP
cr_start()
umap_model = UMAP(n_neighbors=30, 
                  n_components=2, #5
                  min_dist=0.0, 
                  #metric='hellinger',  #cosine
                  random_state=42)# Initiate BERTopic
runtime_umap = cr_getrt()
print(f"umap done {runtime_umap}")

cr_start()
#topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=False)# Run BERTopic model
topic_model = BERTopic(language="english", calculate_probabilities=False)# Run BERTopic model
runtime_topic = cr_getrt()
print(f"bertopic done {runtime_topic}")

cr_start()
topics, probabilities = topic_model.fit_transform(testset[testset['topics'].notna()]['topics'])
runtime_fittr = cr_getrt()
print(f"fitting done {runtime_fittr}")

umap done 0.00011205673217773438
bertopic done 0.00011324882507324219
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment varia

In [43]:
to_disk(topics, f'bart_res_{t_name}_topics')
to_disk(probabilities, f'bart_res_{t_name}_probabilities')
to_disk(topic_model, f'bart_res_{t_name}_topic_model')

/home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/bart_res_t2.3.1s_topics.pkl


AttributeError: 'list' object has no attribute 'to_pickle'

In [14]:
testmodel = from_disk(f'bart_res_{t_name}_topic_model', subdir='', t='bert')
testtopics = from_disk(f'bart_res_{t_name}_topics', subdir='results', t='list')
testprob = from_disk(f'bart_res_{t_name}_probabilities', subdir='results', t='list')

NameError: name 'from_disk_results' is not defined

In [10]:
len(topics)

with pd.option_context('display.max_rows', 1000, 'display.max_columns', 20, 'display.max_colwidth', 1000):
    display(identified_repos_topics.groupby('cluster_run').size())

NameError: name 'topics' is not defined

In [11]:
# Get top 10 terms for a topic
topic_model.get_topic(10)

ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

In [ ]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [ ]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=40)

In [ ]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

In [ ]:
# Visualize probability distribution
topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.015)

In [ ]:
from collections import defaultdict

def max_fscore(d, dbg=False):
    #max_fscore(results[[tag, 'cluster_run']].apply(tuple, axis=1).to_dict()
    # key: (my_cluster, kmeans_cluster)
    #d = {'a': (0, 0), 'b': (0, 1), 'c': (1, 1), 'd': (1, 3), 'e': (2, 2), 'f': (2, 2), 'g': (2, 0)}
    #d = {'a': (0, 10), 'b': (0, 10), 'c': (1, 11), 'd': (1, 11), 'e': (1, 12), 'f': (2, 13), 'g': (3, 13)}
    #d = {'a': (0, 100), 'b': (0, 100), 'c': (1, 101)}
        
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    # Double loop to compare each repo with every other repo
    already_processed = []
    for outer in d.keys():
        already_processed.append(outer)
        for inner in d.keys():
            if inner in already_processed:
                # Repos are not compared to themselves and each repo should only be compared with each other repo once (i.e. a->b then b->a is not necessary anymore)
                continue
            my_cluster_outer = d[outer][0]
            kmeans_outer = d[outer][1]
            my_cluster_inner = d[inner][0]
            kmeans_inner = d[inner][1]
            #kmeans_inner_list = tag_list[my_cluster_inner]
            if kmeans_outer == kmeans_inner: #kmeans_outer == kmeans_inner: # if inner clusterlabel == outer clusterlabel
                # outer and inner were clustered into the same cluster
                if my_cluster_outer == my_cluster_inner: # if inner my tag == outer my tag
                    # outer and inner should be in the same cluster
                    tp += 1
                    if dbg: print(outer + '-' + inner + ': TP')
                else:
                    # outer and inner should be in different clusteres
                    fp += 1
                    if dbg: print(outer + '-' + inner + ': FP')
            else:
                # outer and inner were clustered into different clusters
                if my_cluster_outer == my_cluster_inner:
                    # outer and inner should be in the same cluster
                    fn += 1
                    if dbg: print(outer + '-' + inner + ': FN')
                else:
                    tn += 1
                    if dbg: print(outer + '-' + inner + ': TN')
    f1 = (2.0*tp) / (2.0*tp + fp + fn)
    
    try:
        fpr = fp / (fp + tn)
        tnr = 1 - fpr # tn / (tn + fp)
    except ZeroDivisionError as e:
        try:
            tnr = tn / (tn + fp)
            fpr = 1 - tnr
        except ZeroDivisionError as e:
            tnr = 'N/A'
            fpr = 'N/A'
    

    
    tpr = tp / (tp + fn) #recall / hit rate

    ppv = tp / (tp + fp) # precision
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    fnr = fn / (fn + tp) # miss
    
    return { 'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'f1': f1,
            'accuracy': accuracy,
            'ppv': ppv,
            'fpr': fpr,
            'tpr': tpr,
            'tnr': tnr,
            'fnr': fnr
    }

In [ ]:
testset['cluster_run'] = topics

In [ ]:
identified_repos_topics = pd.merge(identified_repos, testset[["name", "cluster_run"]], on="name", how="left")

In [ ]:
results = identified_repos_topics[identified_repos_topics['cluster_run'].notna()]
results = results[results['cluster_run'] != -1]

stats = {'type': "BART",
         'testset': t_name,
         'version': version,
         'size':'N/A'
        }

my_results = []

lo_tags = [c for c in identified_repos.columns if 'tag' in c]
for tag in lo_tags:        
    prediction = stats
    prediction['tag'] = tag
    prediction.update(max_fscore(results[[tag, 'cluster_run']].apply(tuple, axis=1).to_dict()))
    #print(prediction)
    my_results.append(prediction.copy())
    
pd.DataFrame.from_dict(my_results)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

with pd.option_context('display.max_rows', 1000, 'display.max_columns', 20, 'display.max_colwidth', 1000):
    display(identified_repos_topics.sort_values('cluster_run'))